# Installation

The easiest way to install `laue-dials` and its dependencies is using [Anaconda](https://docs.anaconda.com/free/anaconda/install/index.html). First you should update and install the libmamba solver with

```
conda update -n base conda
conda install -n base conda-libmamba-solver
conda config --set solver libmamba
```

With Anaconda, you can then create and activate a custom environment for your install by running  

```
conda create --name laue-dials
conda activate laue-dials
```

Now you are ready to install the main dependency and framework: [DIALS](https://dials.github.io). After installing that, you can install `laue-dials` using pip, as below:

```
conda install -c conda-forge dials
pip install laue-dials
```

All other dependencies will then be automatically installed for you, and you'll be ready to analyze your first Laue data set! Reopen this notebook with the appropriate environment activated when ready.

Documentation for `laue-dials` can be found at [here](https://rs-station.github.io/laue-dials/index.html), and entering a command with no arguments on the command line will also print a help page!

# Importing Data

We can use `dials.import` as a way to import the data files written at experimental facilities into a format that is friendly to both `DIALS` and `laue-dials`. Feel free to use any data set you'd like below, but a [sample lysozyme data set](https://zenodo.org/record/6407157) has been uploaded to zenodo for your convenience, and this notebook has been tested using that dataset.

In [1]:
%%time
%%bash

# Import data
# You may need to change some of these values to match your data set
dials.import geometry.scan.oscillation=0,1 \
    geometry.goniometer.axes=0,1,0 \
    geometry.beam.wavelength=1.04 \
    geometry.detector.panel.pixel=0.08854,0.08854 \
    input.template="/home/rahewitt/data/e080_raw_data/e080_###.mccd"  #"PATH/TO/DATA/e080_###.mccd" # Replace with the path to your raw data

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
DIALS 3.15
The following parameters have been modified:

input {
  template = "/home/rahewitt/data/e080_raw_data/e080_###.mccd"
}
geometry {
  beam {
    wavelength = 1.04
  }
  detector {
    panel {
      pixel_size = 0.08854 0.08854
    }
  }
  goniometer {
    axes = 0 1 0
  }
  scan {
    oscillation = 0 1
  }
}


Applying input geometry in the following order:
  1. Manual geometry

--------------------------------------------------------------------------------
  format: <class 'dxtbx.format.FormatTIFFRayonix.FormatTIFFRayonix'>
  template: /home/rahewitt/data/e080_raw_data/e080_###.mccd:1:179
  num images: 179
  sequences:
    still:    0
    sweep:    1
  num stills: 0
--------------------------------------------------------------------------------
Writing experiments to imported.expt
CPU times: user 16.4 ms, sys: 3.26 ms, total: 19.7 ms
Wall time: 750 ms


# Getting an Initial Estimate

After importing our data, the first thing we need to do is get an initial estimate for the experimental geometry. Here, we'll use some monochromatic algorithms from DIALS to help! This step can be tricky -- failure can be due to several causes. In the event of failure, here are a few common causes:

1. The spotfinding gain is either too high or too low. Try looking at the results of `dials.image_viewer imported.expt strong.refl` and seeing if you have too many (or too few) reflections. Higher gain gives you more spots, but also more likely to give false positives.
2. Supplying the space group or unit cell during indexing can be helpful. When supplying the unit cell, allow for some variation in the lengths of the axes, since the monochromatic algorithms may result in a slightly scaled unit cell depending on the chosen wavelength.
3. You may have intensities that need to be masked. These can come from bad panels or extraneous scatter. You can use `dials.image_viewer` (described below) to create a mask file for your data, and then provide the `spotfinder.lookup.mask="pixels.mask"` command below to use that mask during spotfinding.

In [2]:
%%time
%%bash

laue.initial_solution imported.expt indexer.indexing.known_symmetry.space_group=19 \
                                    indexer.refinement.parameterisation.auto_reduction.action=None \
                                    spotfinder.lookup.mask="pixels.mask"

The following parameters have been modified:

spotfinder {
  spotfinder {
    lookup {
      mask = "pixels.mask"
    }
  }
}
indexer {
  indexing {
    known_symmetry {
      space_group = "P 21 21 21"
    }
  }
  refinement {
    parameterisation {
      auto_reduction {
        action = fail fix remove
      }
    }
  }
}
input {
  experiments = imported.expt
}

The following parameters have been modified:

spotfinder {
  spotfinder {
    lookup {
      mask = "pixels.mask"
    }
  }
}
indexer {
  indexing {
    known_symmetry {
      space_group = "P 21 21 21"
    }
  }
  refinement {
    parameterisation {
      auto_reduction {
        action = fail fix remove
      }
    }
  }
}
input {
  experiments = imported.expt
}


********************************************************************************
Finding strong spots
********************************************************************************
Setting spotfinder.filter.min_spot_size=6
Configuring spot finder from input par

# Viewing Images

Sometimes it's helpful to be able to see the analysis data overlayed on the raw data. DIALS has a utility for viewing spot information on the raw images called `dials.image_viewer`. For example, the spotfinding gain parameter can be tuned to capture more spots, but increasing too much finds nonexistent spots. To check this, we can use the image viewer to see what spots were found on images. We need to provide an `expt` file and a `refl` file -- the `imported.expt` and `strong.refl` files will do for checking spotfinding. This program also has utilities for generating masks if they are needed.

In [4]:
%%time
%%bash

dials.image_viewer imported.expt strong.refl

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
The following parameters have been modified:

input {
  experiments = imported.expt
  reflections = strong.refl
}



Gtk-Message: 13:53:21.403: Failed to load module "canberra-gtk-module"
Gtk-Message: 13:53:21.403: Failed to load module "pk-gtk-module"


CPU times: user 5.21 ms, sys: 1.76 ms, total: 6.98 ms
Wall time: 4.19 s


# Making Stills

Here we will now split our monochromatic estimate into a series of stills to prepare it for the polychromatic pipeline. There is a useful utility called `laue.sequence_to_stills` for this.

NOTE: Do not use `dials.sequence_to_stills`, as there are data columns which do not match between the two programs.

In [5]:
%%time
%%bash

laue.sequence_to_stills monochromatic.*

/home/rahewitt/laue_dials/laue-dials/src/laue_dials/command_line/sequence_to_stills.py:170: UserWarning: OptionParser is deprecated, use ArgumentParser instead
  parser = OptionParser(
Fatal Python error: Bus error

Current thread 0x00007fe7243c0740 (most recent call first):
  File "/home/rahewitt/anaconda3/envs/laue_test/lib/python3.11/site-packages/libtbx/phil/__init__.py", line 1160 in extract_format
  File "/home/rahewitt/anaconda3/envs/laue_test/lib/python3.11/site-packages/libtbx/phil/__init__.py", line 1904 in fetch
  File "/home/rahewitt/anaconda3/envs/laue_test/lib/python3.11/site-packages/libtbx/phil/__init__.py", line 1878 in fetch
  File "/home/rahewitt/anaconda3/envs/laue_test/lib/python3.11/site-packages/libtbx/phil/__init__.py", line 1941 in fetch_diff
  File "/home/rahewitt/anaconda3/envs/laue_test/lib/python3.11/site-packages/dials/util/options.py", line 436 in diff_phil
  File "/home/rahewitt/anaconda3/envs/laue_test/lib/python3.11/site-packages/dials/util/options.py"

CalledProcessError: Command 'b'\nlaue.sequence_to_stills monochromatic.*\n'' returned non-zero exit status 135.

# Polychromatic Analysis

Here we will use four other programs in `laue-dials` to create a polychromatic experimental geometry using our initial monochromatic estimate. Each of the programs does the following:

`laue.optimize_indexing` assigns wavelengths to reflections and refines the crystal orientation jointly.

`laue.refine` is a polychromatic wrapper for `dials.refine` and allows for refining the experimental geometry overall to one suitable for spot prediction and integration.

`laue.predict` takes the refined experimental geometry and predicts the centroids of all strong and weak reflections on the detector.

`laue.integrate` then builds spot profiles and integrates intensities on the detector.

In [ ]:
%%time
%%bash

N=12 # Max multiprocessing
laue.optimize_indexing stills.* output.experiments="optimized.expt" output.reflections="optimized.refl" output.log="laue.optimize_indexing.log" n_proc=$N
laue.refine optimized.* output.experiments="poly_refined.expt" output.reflections="poly_refined.refl" output.log="laue.poly_refined.log" n_proc=$N

## DIALS Reports

DIALS has a utility that gives useful information on various diagnostics you may be interested in while analyzing your data. The program `dials.report` generates an HTML file you can open to see information and plots regarding the status of your analyzed data. You can run it on any files generated by `DIALS` or `laue-dials`. 

In [ ]:
%%time
%%bash

dials.report poly_refined.expt poly_refined.refl

## Integrating Spots

Now that we have a refined experiment model, we can use `laue.predict` and `laue.integrate` to get integrated intensities from the data. We will predict the locations of all feasible spots on the detector given our refined experiment model, and at each of those locations we will integrate the intensities to get an `mtz` file that we can feed into `careless`.

In [ ]:
%%time
%%bash

N=12 # Max multiprocessing
laue.predict poly_refined.* output.reflections="predicted.refl" output.log="laue.predict.log" n_proc=$N
laue.integrate poly_refined.expt predicted.refl output.filename="integrated.mtz" output.log="laue.integrate.log" n_proc=$N

# Conclusion

At this point, you now have an integrated `mtz` file that you can pass to [careless](https://github.com/rs-station/careless) for scaling and merging. Note that throughout this pipeline, you can use DIALS utilities like `dials.image_viewer` or `dials.report` to check progress and ensure your data is being analyzed properly. We recommend regularly checking the analysis by looking at the data on images, which can be done by

`dials.image_viewer FILE.expt FILE.refl`.

These files are generally written as pairs with the same base name, with the exception of combining `imported.expt` + `strong.refl`, or `poly_refined.expt` + `predicted.refl`.

Also note that you can take any program and enter it on the command-line for further help. For example, writing

`laue.optimize_indexing`

will print a help page for the program. You can see all configurable parameters by using 

`laue.optimize_indexing -c`.

This applies to all `laue-dials` command-line programs.

Congratulations! This tutorial is now over. For further questions, feel free to consult documentation or email the [authors](https://pypi.org/project/laue-dials/).